In [1]:

!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): Traceback (most recent call last):
  File "/home/shubham/meta/.venv/bin

In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Ensure the model uses the GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load tokenizer
model_name = "meta-llama/Llama-3.3-70B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load model with memory optimization
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",           # Automatically distribute the model across available GPUs
    torch_dtype=torch.float16,   # Use FP16 to reduce memory footprint
    low_cpu_mem_usage=True       # Reduce CPU RAM usage during loading
)

# Confirm model is on GPU
print(f"Model loaded on {device}")


Loading checkpoint shards: 100%|██████████| 30/30 [00:12<00:00,  2.45it/s]
Some parameters are on the meta device because they were offloaded to the disk.


Model loaded on cuda


In [5]:
from datasets import load_dataset

# Load the dataset
dataset = load_dataset('json', data_files='norwegian_chatbot_data.jsonl')

# Tokenize
def tokenize_function(examples):
    return tokenizer(examples['prompt'], truncation=True, padding='max_length', max_length=256)

tokenized_dataset = dataset.map(tokenize_function, batched=True)


Generating train split: 500 examples [00:00, 24138.21 examples/s]
Map: 100%|██████████| 500/500 [00:00<00:00, 8209.32 examples/s]


In [14]:
from transformers import Trainer, TrainingArguments
# Split the dataset into train and eval (90% train, 10% eval)
split_dataset = tokenized_dataset['train'].train_test_split(test_size=0.1)

from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./norwegian_chatbot_model",
    evaluation_strategy="no",              # Disable evaluation to simplify
    learning_rate=3e-5,
    per_device_train_batch_size=1,         # Small batch size for CPU
    num_train_epochs=2,
    weight_decay=0.01,
    optim="adamw_torch",                   # Optimized for CPU
    no_cuda=True                           # Explicitly disable CUDA usage
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=split_dataset['train']
)

# Start training
trainer.train()


/home/shubham/meta/.venv/lib/python3.10/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/home/shubham/meta/.venv/lib/python3.10/site-packages/transformers/training_args.py:1590: FutureWarning: using `no_cuda` is deprecated and will be removed in version 5.0 of 🤗 Transformers. Use `use_cpu` instead
  warnings.warn(
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


ValueError: The model did not return a loss from the inputs, only the following keys: logits. For reference, the inputs it received are input_ids,attention_mask.

In [16]:
model.gradient_checkpointing_enable() = = false


SyntaxError: cannot assign to function call (1418888031.py, line 1)